In [44]:
from __future__ import print_function

import os
import numpy as np
import nibabel as nib
import skfuzzy as fuzz
from sklearn.cluster import KMeans
from scipy.signal import medfilt
from multiprocessing import Pool, cpu_count

In [51]:
def create_dir(path):
    if not os.path.isdir(path):
        os.makedirs(path)
    return


def load_nii(path):
    nii = nib.load(path)
    return nii.get_data(), nii.get_affine()


def save_nii(data, path, affine):
    nib.save(nib.Nifti1Image(data, affine), path)
    return


def denoise(volume, kernel_size=3):
    return medfilt(volume, kernel_size)


def extract_features(data):
    # find pixel =! 0
    x_idx, y_idx, z_idx = np.where(data > 0)
    features = []
    for x, y, z in zip(x_idx, y_idx, z_idx):
        features.append([data[x, y, z], x, y, z])
    return np.array(features)
    

def kmeans_cluster(data, n_clusters):
    features = extract_features(data)
    intensities = features[..., 0].reshape((-1, 1))
    kmeans_model = KMeans(n_clusters=n_clusters, init="k-means++",
                          precompute_distances=True, verbose=0,
                          random_state=7, n_jobs=1,
                          max_iter=1000, tol=1e-6).fit(intensities)

    labels = np.zeros(data.shape)
    for l, f in zip(kmeans_model.labels_, features):
        labels[int(f[1]), int(f[2]), int(f[3])] = l + 1

    return labels


def get_target_label(labels, data):
    labels_set = np.unique(labels)
    mean_intensities = []
    for label in labels_set[1:]:
        label_data = data[np.where(labels == label)]
        mean_intensities.append(np.mean(label_data))
        
    target_gm = np.median(mean_intensities)  # GM
    target_wm = np.max(mean_intensities)  # WM
    target_csf = np.min(mean_intensities)  # CSF
    
    target_label_gm = mean_intensities.index(target_gm) + 1
    target_label_wm = mean_intensities.index(target_wm) + 1
    target_label_csf = mean_intensities.index(target_csf) + 1
    return target_label_gm, target_label_wm, target_label_csf


def segment(src_path, upper_path, dst_path):
    print("Segment on: ", src_path)
    try:
        data, affine = load_nii(src_path)
        
        n_clusters = 3
        kernel_size = 3
        
        data = denoise(data, kernel_size)

        labels = kmeans_cluster(data, n_clusters)

        target_gm, target_wm, target_csf = get_target_label(labels, data)
        
        gm_mask = np.copy(labels).astype(np.float32)
        gm_mask[np.where(gm_mask != target_gm)] = 0.
        gm_mask[np.where(gm_mask == target_gm)] = 1.
        
        wm_mask = np.copy(labels).astype(np.float32)
        wm_mask[np.where(wm_mask != target_wm)] = 0.
        wm_mask[np.where(wm_mask == target_wm)] = 1.
        
        csf_mask = np.copy(labels).astype(np.float32)
        csf_mask[np.where(csf_mask != target_csf)] = 0.
        csf_mask[np.where(csf_mask == target_csf)] = 1.
        
        data = data.astype(np.float32)
        gm = np.multiply(data, gm_mask)
        wm = np.multiply(data, wm_mask)
        csf = np.multiply(data, csf_mask)
        
        save_nii(labels,upper_path + 'labels_' + dst_path, affine)
        save_nii(gm, upper_path + 'gm_' + dst_path, affine)
        save_nii(wm,upper_path + 'wm_' + dst_path, affine)
        save_nii(csf,upper_path + 'csf_' + dst_path, affine)
        
    except RuntimeError:
        print("\tFalid on: ", src_path)

    return

In [52]:
path = "./"
file_list = os.listdir(path+'strip_img')
for i in file_list:
    segment(path+'strip_img/'+i, 'segment_img/', i)

Segment on:  ./strip_img/NorFT1001.nii


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: get_affine method is deprecated.
Please use the ``img.affine`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  if __name__ == '__main__':


Segment on:  ./strip_img/NorFT1002.nii


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: get_affine method is deprecated.
Please use the ``img.affine`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  if __name__ == '__main__':


Segment on:  ./strip_img/NorFT1003.nii


...

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: get_affine method is deprecated.
Please use the ``img.affine`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  if __name__ == '__main__':


Segment on:  ./strip_img/NorMT1009.nii


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: get_affine method is deprecated.
Please use the ``img.affine`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  if __name__ == '__main__':


Segment on:  ./strip_img/NorMT1010.nii


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: get_affine method is deprecated.
Please use the ``img.affine`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  if __name__ == '__main__':
